<a href="https://colab.research.google.com/github/ctshiz/Spark/blob/main/Spark_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALL SPARK**

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark
!pip install pyspark[sql]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.1 kB]
Get:12 http://archive.ubuntu.com/ubun

In [2]:
!ls

sample_data  spark-3.1.1-bin-hadoop2.7	spark-3.1.1-bin-hadoop2.7.tgz


*****************************************************************************
**SET ENVIRONMENT**

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark =  SparkSession.builder.getOrCreate()
spark

In [4]:
import os
os.environ["JAVA_HOME"] = "/user/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


*****************************************************************************
**DOWNLOADING AND PREPROCESSING CHICAGO'S REPORTED CRIME SCENE**


In [5]:
!wget https://data.cityofchicago.org/api/views/6y6a-5u9n/rows.csv?accessType=DOWNLOAD

--2021-04-27 14:35:57--  https://data.cityofchicago.org/api/views/6y6a-5u9n/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [ <=>                ] 257.71K  --.-KB/s    in 0.09s   

2021-04-27 14:35:58 (2.81 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [263894]



In [6]:
!ls 

'rows.csv?accessType=DOWNLOAD'	 spark-3.1.1-bin-hadoop2.7
 sample_data			 spark-3.1.1-bin-hadoop2.7.tgz


In [7]:
#rename the file into reported crimes
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv

In [9]:
#read the files
from pyspark.sql.functions import to_timestamp, col, lit
rc =  spark.read.csv('reported-crimes.csv', header=True).withColumn('Date', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <  lit('2017-01-12'))
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11711896|   JC293058|2013-06-05 12:00:00|022XX S PRINCETON...|0485|           BATTERY|AGGRAVATED OF A C...|           RESIDENCE| fal

*****************************************************************************
**SCHEMA**

In [10]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [11]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

In [12]:
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [13]:
labels = [
          ('ID', StringType()),
          ('Case Number', StringType()),
          ('Date', TimestampType()),
          ('Block',StringType()),
          ('IUCR', StringType()),
          ('Primary Type',StringType()),
          ('Description',StringType()),
          ('Location Description',StringType()),
          ('Arrest',StringType()),
          ('Domestic', BooleanType()),
          ('Beat', StringType()),
          ('District',StringType()),
          ('Ward',StringType()),
          ('Community Area', StringType()),
          ('FBI Code', StringType()),
          ('X Coordinate', StringType()),
          ('Y Coordinate', StringType()),
          ('Year', IntegerType()),
          ('Updated On', StringType()),
          ('Latitude', DoubleType()),
          ('Longitude', DoubleType()),
          ('Location', StringType())
]

In [14]:
schema = StructType([StructField(x[0], x[1], True) for x in labels])

In [15]:
rc = spark.read.csv('reported-crimes.csv', schema=schema)
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [16]:
rc.show(5)

+--------+-----------+----+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+----+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|null|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|    null|Beat|District|Ward|Community Area|FBI Code|X Coo

*****************************************************************************
**WORKING WITH COLUMNS**

Display only the first 5 roow of the columns name IUCR

In [17]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|IUCR|
|0485|
|1812|
|1154|
|1811|
+----+
only showing top 5 rows



In [18]:
rc.select(col('IUCR')).show(5)

+----+
|IUCR|
+----+
|IUCR|
|0485|
|1812|
|1154|
|1811|
+----+
only showing top 5 rows



Display only the first 4 rows of the colum names Case Number, Date and Arrest

In [19]:
rc.select('Case Number', 'Date', 'Arrest').show(4)

+-----------+----+------+
|Case Number|Date|Arrest|
+-----------+----+------+
|Case Number|null|Arrest|
|   JC293058|null| false|
|   HW130127|null|  true|
|   HY304097|null| false|
+-----------+----+------+
only showing top 4 rows



Add a column with name One, with entries all 1s

In [20]:
from pyspark.sql.functions import lit

In [21]:
rc.withColumn('One', lit(1)).show(5)

+--------+-----------+----+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|      ID|Case Number|Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|One|
+--------+-----------+----+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|      ID|Case Number|null|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|    null|Beat|District|Ward|Community Area|FB

Remove the column IUCR

In [22]:
rcp = rc.drop('IUCR')
rcp.show(5)

+--------+-----------+----+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|Date|               Block|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+----+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|null|               Block|      Primary Type|         Description|Location Description|Arrest|    null|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate

*****************************************************************************
**WORKING WITH ROWS**

Add the reported crimes for an additional day, 12-nov-2018, to our dataset

In [23]:
one_day = spark.read.csv('reported-crimes.csv', header=True).withColumn('Date', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <  lit('2015-01-12'))
one_day.show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11711896|   JC293058|2013-06-05 12:00:00|022XX S PRINCETON...|0485|           BATTERY|AGGRAVATED OF A C...|           RESIDENCE| fal

TOP 10 NUMBER OF REPORTED CRIMES BY PRIMARY TYPE, IN DESCENDING ORDER OF OCCURENCE

In [24]:
rc.groupBy('Primary Type').count().orderBy('count', ascending=False).show(10)

+-------------------+-----+
|       Primary Type|count|
+-------------------+-----+
|              THEFT|  396|
|    CRIMINAL DAMAGE|  141|
|            BATTERY|  117|
|MOTOR VEHICLE THEFT|  102|
| DECEPTIVE PRACTICE|   88|
|           BURGLARY|   85|
|            ROBBERY|   52|
|      OTHER OFFENSE|   37|
|          NARCOTICS|   29|
|            ASSAULT|   29|
+-------------------+-----+
only showing top 10 rows



*****************************************************************************
**CHALLENGE QUESTIONS**

THE PERCENTAGE OF REPORTED CRIMES RESULTED IN AN ARREST

In [25]:
rc.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
|Arrest|
| false|
|  true|
+------+



In [26]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [27]:
rc.filter(col('Arrest') == 'true').count() / rc.select('Arrest').count()

0.15575221238938053

TOP 3 LOCATIONS FOR REPORTED CRIMES

In [28]:
rc.groupBy('Location Description').count().orderBy('count', ascending=False).show(3)

+--------------------+-----+
|Location Description|count|
+--------------------+-----+
|              STREET|  435|
|           RESIDENCE|  109|
|           APARTMENT|  103|
+--------------------+-----+
only showing top 3 rows



**BUILT-IN FUNCTIONS**

In [29]:
from pyspark.sql import functions
print(dir(functions))

['Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_column_from_literal', '_create_lambda', '_create_udf', '_get_get_jvm_function', '_get_lambda_parameters', '_invoke_binary_math_function', '_invoke_function', '_invoke_function_over_column', '_invoke_higher_order_function', '_options_to_str', '_test', '_to_java_column', '_to_seq', '_unresolved_named_lambda_variable', 'abs', 'acos', 'acosh', 'add_months', 'aggregate', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'array_distinct', 'array_except', 'array_intersect', 'array_join', 'array_max', 'array_min', 'array_position', 'array_remove', 'array_repeat', 'array_sort', 'array_union', 'arrays_overlap', 'arrays_zip', 'asc', 'asc_nulls_first', 'asc_nulls_last', 'ascii', 'asin', 'asinh', 'assert_true', 'atan', 'atan2', 'atanh', 'a

STRING FUNCTIONS

Display the primary type column in lower and upper characters, and the first 4 characters 

In [30]:
from pyspark.sql.functions import lower,upper, substring

In [31]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [32]:
rc.select(lower(col('Primary Type')), upper(col('Primary Type')), substring(col('Primary Type'),1,4)).show(5)

+-------------------+-------------------+-----------------------------+
|lower(Primary Type)|upper(Primary Type)|substring(Primary Type, 1, 4)|
+-------------------+-------------------+-----------------------------+
|       primary type|       PRIMARY TYPE|                         Prim|
|            battery|            BATTERY|                         BATT|
|          narcotics|          NARCOTICS|                         NARC|
| deceptive practice| DECEPTIVE PRACTICE|                         DECE|
|          narcotics|          NARCOTICS|                         NARC|
+-------------------+-------------------+-----------------------------+
only showing top 5 rows



NUMERIC FUNCTIONS

In [33]:
from pyspark.sql.functions import min,max

In [34]:
one_day.select(min(col('Date')), max(col('Date'))).show(2)

+-------------------+-------------------+
|          min(Date)|          max(Date)|
+-------------------+-------------------+
|2012-10-14 00:16:00|2013-10-13 22:00:00|
+-------------------+-------------------+



DATE

In [35]:
from pyspark.sql.functions import date_add, date_sub

In [36]:
help(date_add)

Help on function date_add in module pyspark.sql.functions:

date_add(start, days)
    Returns the date that is `days` days after `start`
    
    .. versionadded:: 1.5.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_add(df.dt, 1).alias('next_date')).collect()
    [Row(next_date=datetime.date(2015, 4, 9))]



In [38]:
one_day.select(date_sub(min(col('Date')),3), date_add(max(col('Date')),3)).show(1)

+----------------------+----------------------+
|date_sub(min(Date), 3)|date_add(max(Date), 3)|
+----------------------+----------------------+
|            2012-10-11|            2013-10-16|
+----------------------+----------------------+



In [39]:
from pyspark.sql.functions import  to_date, to_timestamp, lit

2019-12-25 13:30:00

In [40]:
df = spark.createDataFrame([('2019-12-25 13:30:00',)], ['Christmas'])
df.show(1)

+-------------------+
|          Christmas|
+-------------------+
|2019-12-25 13:30:00|
+-------------------+



In [41]:
df.select(to_date(col('Christmas'), 'yyyy-MM-dd HH:mm:ss'), to_timestamp(col('Christmas'), 'yyyy-MM-dd HH:mm:ss')).show(1)

+---------------------------------------+--------------------------------------------+
|to_date(Christmas, yyyy-MM-dd HH:mm:ss)|to_timestamp(Christmas, yyyy-MM-dd HH:mm:ss)|
+---------------------------------------+--------------------------------------------+
|                             2019-12-25|                         2019-12-25 13:30:00|
+---------------------------------------+--------------------------------------------+



25/dec/2019 13:30:00

In [42]:
df = spark.createDataFrame([('25/Dec/2019 13:30:00',)], ['Christmas'])
df.show(1)

+--------------------+
|           Christmas|
+--------------------+
|25/Dec/2019 13:30:00|
+--------------------+



In [43]:
df.select(to_date(col('Christmas'), 'dd/MMM/yyyy HH:mm:ss'), to_timestamp(col('Christmas'), 'dd/MMM/yyyy HH:mm:ss')).show(1)

+----------------------------------------+---------------------------------------------+
|to_date(Christmas, dd/MMM/yyyy HH:mm:ss)|to_timestamp(Christmas, dd/MMM/yyyy HH:mm:ss)|
+----------------------------------------+---------------------------------------------+
|                              2019-12-25|                          2019-12-25 13:30:00|
+----------------------------------------+---------------------------------------------+



In [44]:
!wget -O police-station.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
!ls -l

--2021-04-27 14:47:10--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.140.205, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘police-station.csv’

police-station.csv      [ <=>                ]   5.57K  --.-KB/s    in 0s      

2021-04-27 14:47:10 (462 MB/s) - ‘police-station.csv’ saved [5699]

total 219392
-rw-r--r--  1 root root      5699 Aug 19  2019 police-station.csv
-rw-r--r--  1 root root    263894 Apr 27 11:23 reported-crimes.csv
drwxr-xr-x  1 root root      4096 Apr 21 13:39 sample_data
drwxr-xr-x 13 1000 1000      4096 Feb 22 02:44 spark-3.1.1-bin-hadoop2.7
-rw-r--r--  1 root root 224374704 Feb 22 02:45 spark-3.1.1-bin-hadoop2.7.tgz


In [45]:
ps = spark.read.csv('police-station.csv', header=True)
ps.show(5)

+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|Headquarters| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|          18|   Near North| 1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 1172080.029| 1908086.527|41.90324165|-87.64335214|(41.9032416531, -

In [46]:
one_day.cache()
one_day.count()

1129

In [47]:
ps.select(col('DISTRICT')).distinct().show(30)

+--------------------+
|            DISTRICT|
+--------------------+
|                   7|
|                  15|
|                  11|
|                   3|
|                   8|
|                  22|
|                  16|
|                   5|
|                  18|
|                  17|
|                   6|
|                  19|
|                  25|
|        Headquarters|
|                  24|
|                   9|
|                   1|
|                  20|
|                  10|
|                   4|
|                  12|
|                  14|
|                   2|
|",Chicago,IL,6060...|
+--------------------+



In [48]:
rc.select('FBI Code').distinct().show(30)

+--------+
|FBI Code|
+--------+
|      07|
|      15|
|      11|
|     08B|
|      22|
|      16|
|      18|
|     04B|
|      17|
|      26|
|      05|
|      03|
|      02|
|     04A|
|      06|
|      24|
|      20|
|      10|
|FBI Code|
|      14|
|     08A|
+--------+



In [49]:
!wget -O community-area.csv https://data.cityofchicago.org/api/views/igwz-8jzy/rows.csv?accessType=DOWNLOAD
!ls

--2021-04-27 14:47:42--  https://data.cityofchicago.org/api/views/igwz-8jzy/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘community-area.csv’

community-area.csv      [        <=>         ]   1.92M  1.20MB/s    in 1.6s    

2021-04-27 14:47:44 (1.20 MB/s) - ‘community-area.csv’ saved [2009107]

community-area.csv  reported-crimes.csv  spark-3.1.1-bin-hadoop2.7
police-station.csv  sample_data		 spark-3.1.1-bin-hadoop2.7.tgz


In [50]:
ca = spark.read.csv('community-area.csv', header=True)
ca.show(5)

+--------------------+---------+----+--------+----------+----------+---------------+----------+-------------+-------------+
|            the_geom|PERIMETER|AREA|COMAREA_|COMAREA_ID|AREA_NUMBE|      COMMUNITY|AREA_NUM_1|   SHAPE_AREA|    SHAPE_LEN|
+--------------------+---------+----+--------+----------+----------+---------------+----------+-------------+-------------+
|MULTIPOLYGON (((-...|        0|   0|       0|         0|        35|        DOUGLAS|        35|46004621.1581|31027.0545098|
|MULTIPOLYGON (((-...|        0|   0|       0|         0|        36|        OAKLAND|        36|16913961.0408|19565.5061533|
|MULTIPOLYGON (((-...|        0|   0|       0|         0|        37|    FULLER PARK|        37|19916704.8692|25339.0897503|
|MULTIPOLYGON (((-...|        0|   0|       0|         0|        38|GRAND BOULEVARD|        38|48492503.1554|28196.8371573|
|MULTIPOLYGON (((-...|        0|   0|       0|         0|        39|        KENWOOD|        39|29071741.9283|23325.1679062|
+-------

In [51]:
ca.select('AREA_NUMBE').distinct().show()

+----------+
|AREA_NUMBE|
+----------+
|        51|
|         7|
|        15|
|        54|
|        11|
|        29|
|        69|
|        42|
|        73|
|        64|
|         3|
|        30|
|        34|
|        59|
|         8|
|        22|
|        28|
|        35|
|        16|
|        52|
+----------+
only showing top 20 rows



In [52]:
rc.select('Community Area').distinct().show()

+--------------+
|Community Area|
+--------------+
|            34|
|            35|
|            60|
|Community Area|
|            33|
+--------------+



In [53]:
ca.select('AREA_NUMBE').count()

77

In [54]:
rc.join(ca, rc(col('Community Area')) == ca.AREA_NUMBE, 'left_outer').show()

TypeError: ignored

THE MOST FREQUENTLY REPORTED NON-CRIMINAL ACTIVITY

In [55]:
rc.cache()
rc.count()

1130

In [56]:
rc.select(col('Primary Type')).distinct().count()

21

In [57]:
rc.select(col('Primary Type')).distinct().orderBy(col('Primary Type')).show(20, truncate=False)

+--------------------------+
|Primary Type              |
+--------------------------+
|ASSAULT                   |
|BATTERY                   |
|BURGLARY                  |
|CRIM SEXUAL ASSAULT       |
|CRIMINAL DAMAGE           |
|CRIMINAL TRESPASS         |
|DECEPTIVE PRACTICE        |
|INTIMIDATION              |
|LIQUOR LAW VIOLATION      |
|MOTOR VEHICLE THEFT       |
|NARCOTICS                 |
|OBSCENITY                 |
|OFFENSE INVOLVING CHILDREN|
|OTHER OFFENSE             |
|PROSTITUTION              |
|PUBLIC PEACE VIOLATION    |
|Primary Type              |
|ROBBERY                   |
|SEX OFFENSE               |
|THEFT                     |
+--------------------------+
only showing top 20 rows



In [58]:
rc.groupBy(col('Description')).count().orderBy('count', ascending=False).show()

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|           OVER $500|  172|
|      $500 AND UNDER|  166|
|          AUTOMOBILE|   94|
|          TO VEHICLE|   92|
|              SIMPLE|   84|
|      FORCIBLE ENTRY|   63|
|THEFT OF LABOR/SE...|   50|
|DOMESTIC BATTERY ...|   43|
|         TO PROPERTY|   37|
|STRONGARM - NO WE...|   33|
|       FROM BUILDING|   27|
|      UNLAWFUL ENTRY|   19|
|    TELEPHONE THREAT|   15|
|             TO LAND|   13|
|      POCKET-PICKING|   11|
|   CREDIT CARD FRAUD|   11|
| CRIMINAL DEFACEMENT|   10|
|FRAUD OR CONFIDEN...|    9|
|POSS: CANNABIS 30...|    9|
|      ARMED: HANDGUN|    8|
+--------------------+-----+
only showing top 20 rows



****************************************************************************
**RDD**

In [59]:
cardd = sc.textFile('community-area.csv')
cardd.first()

'the_geom,PERIMETER,AREA,COMAREA_,COMAREA_ID,AREA_NUMBE,COMMUNITY,AREA_NUM_1,SHAPE_AREA,SHAPE_LEN'

In [60]:
ca_header = cardd.first()

In [61]:
ca_rest = cardd.filter(lambda line: line!= ca_header)
ca_rest.first()

'"MULTIPOLYGON (((-87.60914087617894 41.84469250265398, -87.60914874757808 41.84466159842403, -87.6091611204126 41.84458961193954, -87.60916766215838 41.84451717732316, -87.60916860600166 41.844456260738305, -87.60915012199398 41.84423871659811, -87.60907241249289 41.844194738881015, -87.60900627147821 41.84410646928696, -87.6089650217216 41.84404345755115, -87.60891566390615 41.84395529375054, -87.60889980118988 41.84387361649532, -87.60886701371862 41.84380438280048, -87.6088514342449 41.843697606960866, -87.60881089281094 41.84357184776641, -87.60877127222787 41.84336451715353, -87.6087215608253 41.84330772696518, -87.6086722038968 41.8432195630558, -87.60858152789015 41.84307466210119, -87.60847385682872 41.84294847915887, -87.60839135990155 41.842822455284654, -87.60826740295646 41.842652243366025, -87.6081767282837 41.8425073420888, -87.60806913018423 41.84237488419488, -87.60801956189378 41.84230554400334, -87.60802097919012 41.84218004974061, -87.60799814396883 41.8419728245406

In [62]:
ca_rest.map(lambda line: line.split(',')).count()

77